In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_ollama import ChatOllama
from langchain.chains import create_retrieval_chain

In [20]:
loader = TextLoader(file_path="math.txt",encoding="utf-8")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()
model = ChatOllama(model="llama3:8b")

prompt = ChatPromptTemplate.from_template("""Ответь на следующий вопрос на русском языке, опираясь на предоставленный документ:

<документ>
{context}
</документ>

Вопрос: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [21]:
import os

import telebot

with open('token.txt', 'r') as tokenfile:
    BOT_TOKEN = tokenfile.read()
bot = telebot.TeleBot(BOT_TOKEN)

In [22]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Добрый день! Я готов помочь вам в подготовке к экзамену по введению в мат. анализ! Задайте любой вопрос")

In [23]:
@bot.message_handler(func=lambda msg: True)
def echo_all(message):
    response = retrieval_chain.invoke({"input": message.text})
    bot.reply_to(message, response["answer"])

In [24]:
bot.infinity_polling()

2025-01-25 18:03:17,466 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-01-25 18:03:17,467 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
